<a href="https://colab.research.google.com/github/restrepo/lens/blob/master/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lens
Example of lens search. Check aginst:

https://www.lens.org/lens/scholar/search/results?q=lens_id:009-947-141-343-381

In [0]:
import pandas as pd

In [0]:
ln=pd.read_json('https://raw.githubusercontent.com/restrepo/lens/master/data/lens.json') 

In [3]:
ln.columns

Index(['_id', 'abstract', 'authors', 'chemicals', 'citation_ids',
       'clinical_trials', 'date_published', 'fields_of_study', 'funding',
       'issue', 'journal', 'keywords', 'languages', 'mesh_terms', 'pages_end',
       'pages_start', 'publication_supplementary_type', 'publication_type',
       'record_lens_id', 'referenced_by_count', 'referenced_by_patent_count',
       'references', 'source_urls', 'title', 'volume', 'year_published'],
      dtype='object')

## Explanation of each columns and its several parts
Nested jsons are extracted as new columns if necessary, e.g DOIs, etc

In [0]:
i=0

Article identification in several data bases:

In [4]:
ln.citation_ids.apply(len)

0    3
Name: citation_ids, dtype: int64

In [8]:
ln.citation_ids[0]

[{'type': 'core', 'value': 'core45448247'},
 {'type': 'doi', 'value': '10.1103/physrevd.93.015012'},
 {'type': 'magid', 'value': 'mag2270814667'}]

In [0]:
ln['magid']=ln.citation_ids.apply(lambda x: [y.get('value') for y in x if y.get('type')=='magid']).str[0]
ln['doi']=ln.citation_ids.apply(lambda x: [y.get('value') for y in x if y.get('type')=='doi']).str[0]
ln['core']=ln.citation_ids.apply(lambda x: [y.get('value') for y in x if y.get('type')=='core']).str[0]

In [20]:

magid=ln.fillna('').loc[i,'magid'].split('mag')[-1]
doi=ln.fillna('').loc[i,'doi']
core=ln.fillna('').loc[i,'core']
if magid:
    print('Microsoft academic link for article: https://academic.microsoft.com/#/detail/{}'.format(magid))
if doi:
    print('DOI:                                 https://oadoi.org/{}'.format(doi))
if doi:
    print('core:                                https://core.ac.uk/display/{}'.format(core))    

Microsoft academic link for article: https://academic.microsoft.com/#/detail/2270814667
DOI:                                 https://oadoi.org/10.1103/physrevd.93.015012
core:                                https://core.ac.uk/display/core45448247


Scholarly Citations:

In [24]:
ln.loc[i,'referenced_by_count']

14

Patent Citations

In [25]:
ln.loc[i,'referenced_by_patent_count']

0

In [27]:
ln.loc[i,'references']

[{'lens_id': 157243461017},
 {'lens_id': 511908356197},
 {'lens_id': 554765889905},
 {'lens_id': 625262539586},
 {'lens_id': 656423764547},
 {'lens_id': 1326319377136},
 {'lens_id': 1514844071549},
 {'lens_id': 2493872779822},
 {'lens_id': 2494092339066},
 {'lens_id': 2566123636854},
 {'lens_id': 2825915349771},
 {'lens_id': 3277397450819},
 {'lens_id': 3498731332786},
 {'lens_id': 3596585624382},
 {'lens_id': 3610408284260},
 {'lens_id': 3637284790067},
 {'lens_id': 3759492979011},
 {'lens_id': 4081061587669},
 {'lens_id': 4131186365653},
 {'lens_id': 4231001148730},
 {'lens_id': 4807770246664},
 {'lens_id': 4828167432673},
 {'lens_id': 4836482296561},
 {'lens_id': 5007459714612},
 {'lens_id': 5141667571468},
 {'lens_id': 5568654687238},
 {'lens_id': 5723578187014},
 {'lens_id': 5776840922724},
 {'lens_id': 6191093647811},
 {'lens_id': 6400858984624},
 {'lens_id': 6600218844479},
 {'lens_id': 6692074167290},
 {'lens_id': 6918307828900},
 {'lens_id': 7006910898580},
 {'lens_id': 743654

In [28]:
ln.loc[i,'record_lens_id']

'034-341-431-911-594'